In [1]:
import torch
from torchvision import datasets, models, transforms
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support, roc_auc_score, log_loss
import matplotlib.pyplot as plt
import numpy as np

# Define transformations for the training and test data
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load the datasets with ImageFolder
image_datasets = {x: datasets.ImageFolder("/kaggle/input/seizures-non-seizures/Data/"+x, data_transforms[x]) for x in ['train', 'test']}

# Using the image datasets, define the dataloaders
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32, shuffle=True) for x in ['train', 'test']}

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
model_name = 'densenet'

print("--------- Running {} ---------".format(model_name))

if model_name == 'resnet':
    model = models.resnet50(pretrained=True)
    model.fc = torch.nn.Linear(model.fc.in_features, 2)

elif model_name == 'efficientnet':
    !pip install efficientnet_pytorch
    from efficientnet_pytorch import EfficientNet
    model = EfficientNet.from_pretrained('efficientnet-b0')
    model._fc = torch.nn.Linear(model._fc.in_features, 2)
    
elif model_name == 'densenet':
    import torchvision.models as models
    model = models.densenet121(pretrained=True)
    model.classifier = torch.nn.Linear(model.classifier.in_features, 2)

else:
    raise Exception("Not a valid model name!") 
    
# Move the model to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define a loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

--------- Running densenet ---------


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 91.4MB/s]


In [3]:
class Early_stopping():
    def __init__(self,min_delta=3, tolerance=3):
        self.counter = 0
        self.val_loss = []
        self.min_delta=min_delta
        self.tolerance=tolerance
        self.status = True
        
    def run(self, loss):
        self.val_loss.append(loss)
        
        if len(self.val_loss) < 2:
            self.status = True
            return False
        diff = self.val_loss[-2] - self.val_loss[-1]
        if diff >= self.min_delta:
            self.status = True
            return False
        else:
            self.counter += 1
        if self.counter >= self.tolerance:
            self.status = False
            return True
        else:
            self.status = True
            return False

In [4]:
early_stopping = Early_stopping(0.03,3)
num_epochs = 30
# Train the model
for epoch in range(num_epochs):
    if not early_stopping.status:
        print(" ------ Early stopped! ------ ")
        break
        
    print(f'Epoch {epoch}/{num_epochs - 1}')
    print('-' * 10)

    # Each epoch has a training and validation phase
    for phase in ['train', 'test']:
        if phase == 'train':
            model.train()  # Set model to training mode
        else:
            model.eval()   # Set model to evaluate mode

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(image_datasets[phase])
        epoch_acc = running_corrects.double() / len(image_datasets[phase])

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        
        if phase == 'test':
            if early_stopping.run(epoch_loss):
              break

Epoch 0/29
----------
train Loss: 0.1549 Acc: 0.9321
test Loss: 1.4764 Acc: 0.7750
Epoch 1/29
----------
train Loss: 0.0987 Acc: 0.9607
test Loss: 0.7360 Acc: 0.7458
Epoch 2/29
----------
train Loss: 0.0310 Acc: 0.9875
test Loss: 0.0871 Acc: 0.9750
Epoch 3/29
----------
train Loss: 0.0280 Acc: 0.9857
test Loss: 0.1395 Acc: 0.9667
Epoch 4/29
----------
train Loss: 0.0344 Acc: 0.9911
test Loss: 0.1294 Acc: 0.9750
Epoch 5/29
----------
train Loss: 0.0132 Acc: 0.9964
test Loss: 0.0881 Acc: 0.9583
Epoch 6/29
----------
train Loss: 0.0063 Acc: 0.9982
test Loss: 0.1257 Acc: 0.9750
 ------ Early stopped! ------ 


In [5]:
from PIL import Image
def Predict(img_path):
    # Load the image
    img = Image.open(img_path)

    # Define the same transformations as before
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # Apply the transformations and add a batch dimension
    img = transform(img).unsqueeze(0)

    # Move the image to the device
    img = img.to(device)

    # Set the model to evaluation mode
    model.eval()

    # Forward pass
    with torch.no_grad():
        output = model(img)

    # Get the predicted class
    _, pred_label = torch.max(output, 1)

    return pred_label.item()

In [6]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support
import numpy as np
import glob, os

# Assume test_data is your test dataset and test_labels are the true labels
# test_data and test_labels should be defined in your code

# Initialize lists to gather true and predicted labels
y_true = []
y_pred = []

test_data  = glob.glob(os.path.join("/kaggle/input/seizures-non-seizures/Data/test/None_seizures", f'Mask_image_None_seizures_*.png'))
test_labels = [0]*len(test_data)

test_data += glob.glob(os.path.join("/kaggle/input/seizures-non-seizures/Data/test/Seizures", f'Mask_image_Seizures_*.png'))
test_labels += [1]*(len(test_data)-len(test_labels))

# Iterate over the test data
for i in range(len(test_data)):
    img = test_data[i]
    label = test_labels[i]

    # Use the Predict function to get the predicted label
    pred_label = Predict(img)

    # Append the true and predicted labels to the lists
    y_true.append(label)
    y_pred.append(pred_label)

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Compute metrics
print(f'Accuracy: {accuracy_score(y_true, y_pred):.3f}')
print(f'Confusion Matrix:\n {confusion_matrix(y_true, y_pred)}')
print("Precision, Recall, F1-score: {:.3f} - {:.3f} - {:.3f}".format(*precision_recall_fscore_support(y_true, y_pred, average="binary")))

Accuracy: 0.975
Confusion Matrix:
 [[118   2]
 [  4 116]]
Precision, Recall, F1-score: 0.983 - 0.967 - 0.975
